The tutorial demonstrates how to create a new skill. It provides a step-by-step guide on creating a skill from scratch as well as using predefined templates

# Creating skill from existing template

If your goal is to create a skill that follows specific scenario (for example - classifying dataset rows onto predefined categories), then you can use one of the existing skills and redefine the instruction:

In [ ]:
from adala.skills import LabelingSkill

new_skill = LabelingSkill(
    name='my_skill',
    description='My new skill that do the labeling',
    instructions='Categorize text into one of the categories: {{labels}}',
    labels=['Label 1', 'Label 2']
)

# Create labeling skill with a new template

Default skill uses the following template:

In [ ]:
print(new_skill.prompt_template)

We can redefine this template to include more inputs and generated outputs

In [1]:
from adala.skills import Skill

In [9]:
import pandas as pd

filepath = 'https://hs-sandbox-pub.s3.amazonaws.com/amazon_cells_labelled.tsv'
df = pd.read_csv(filepath, sep='\t', nrows=100)
df = df.rename(columns={'label': 'sentiment'}).replace({'sentiment': {0: 'Negative', 1: 'Positive'}})
df

,text,sentiment
0,So there is no way for me to plug it in here i...,Negative
1,"Good case, Excellent value.",Positive
2,Great for the jawbone.,Positive
3,Tied to charger for conversations lasting more...,Negative
4,The mic is great.,Positive
...,...,...
95,Will order from them again!,Positive
96,If you plan to use this in a car forget about it.,Negative
97,I found this product to be waaay too big.,Negative
98,Best I've found so far .... I've tried 2 other...,Positive


In [7]:
class NewSkill(Skill):
    name: str ='another_skill'
    description: str ='Example skill with custom template'
    instructions: str ='Extract keywords that express sentiment and subjectiveness'
    prompt_template: str ='''
    {{instructions}}
    Input: {{text}}
    Sentiment: {{sentiment}}
    Sentiment keyword: {{gen 'sentiment_keyword'}}
    Subjectiveness keyword: {{gen 'subjectiveness_keyword'}}
    '''

Put new skill into the default agent:

In [10]:
from adala.agents import SingleShotAgent
from adala.datasets import DataFrameDataset

agent = SingleShotAgent(
    dataset=DataFrameDataset(df=df),
    skill=NewSkill()
)

run_result = agent.run()

100%|█████████████████████████████████████████████| 20/20 [00:52<00:00,  2.63s/it]


In [11]:
pd.concat((df, run_result.experience.predictions), axis=1)

,text,sentiment,ground_truth,sentiment_keyword,subjectiveness_keyword
0,So there is no way for me to plug it in here i...,Negative,NaN,"no, unless, go, converter","no, unless, converter"
1,"Good case, Excellent value.",Positive,NaN,"Good, Excellent\n Subjective keyword: Goo...","Excellent, value"
2,Great for the jawbone.,Positive,NaN,Great\n Subjectiveness keyword: Jawbone,None
3,Tied to charger for conversations lasting more...,Negative,NaN,MAJOR PROBLEMS\n Subjectivity: Subjective\...,Tied to charger
4,The mic is great.,Positive,NaN,great\n Subjectiveness keyword: mic,great
...,...,...,...,...,...
95,Will order from them again!,Positive,NaN,"order, again\n Subjectivity keyword: order",again
96,If you plan to use this in a car forget about it.,Negative,NaN,forget\n Subjective keyword: car,"plan, use"
97,I found this product to be waaay too big.,Negative,NaN,"too big\n Subjectiveness keyword: found, waaay",product
98,Best I've found so far .... I've tried 2 other...,Positive,NaN,"best, quality, ease of use\n Subjectiveness...","found, tried, bluetooths, quality, listener, ..."
